In [ ]:
import time, scipy
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import wasserstein_distance
import scipy.signal
import time
import freeDeconvolution
#import subordination, sampling, elkaroui
import multiprocessing as mp
import json

T = np.arange(0,10, 0.2)
nu = np.array( [complex(0.1*a,b) for a in range(0  , 10) for b in [0.01,0.1]] )

# Returns Scenario in the form of p eigenvalues
def gen_matrix(Scenario, p):
        np.random.seed(seed=None)
        if Scenario=="Case1":
            population_spectrum = np.ones( p )
            weights = np.array( [1], dtype=np.float64 )
            support = np.array( [1], dtype=np.float64 )

        elif Scenario=="Case2" or Scenario=="Case2.2" or Scenario=="Case2.3" :
            if Scenario=="Case2":
                weights = np.array( [1, 1], dtype=np.float64 )
                support = np.array( [1, 2], dtype=np.float64 )
            elif Scenario=="Case2.2" :
                weights = np.array( [1, 1], dtype=np.float64 )
                support = np.array( [1, 1.3], dtype=np.float64 )
            else :
                weights = np.array( [1, 1, 1, 1, 1], dtype=np.float64 )
                support = np.array( [1, 2, 3, 5, 6], dtype=np.float64 )
            weights = weights/np.sum( weights )
 
            population_cdf = np.cumsum( weights )

            population_spectrum = np.zeros( (p,) )
            block_begin = 0
            for i in range( len(weights) ):
                block_end = int( population_cdf[i]*p )
                population_spectrum[block_begin:block_end] = support[i]
                block_begin = block_end

        elif Scenario=="Case3":
            c = 1
            indices = np.arange( 0, p, 1)
            toeplitz_row    = 0.3**indices
            toeplitz = scipy.linalg.toeplitz( toeplitz_row)
            
            population_spectrum, U = np.linalg.eig(toeplitz)
            population_spectrum = np.sort( population_spectrum )

            weights = np.ones(p)*1.0/p
            support = population_spectrum

        else:
            print( "Please specify a scenario..." )
            raise Error()
        
        return (population_spectrum, weights, support)

# I. Generate experiments JSON

In [ ]:
N_range   = [ 1024 ]
nb_itr    = 1
#Scenarios = [ "Case1", "Case2", "Case2.2", "Case2.3", "Case3"]
Scenarios = [ "Case2.3"]

DOEs = []

# Loop over scenarios
for Scenario in Scenarios:
    print( f'''  |- Scenario {Scenario}''')
    # Loop over iterations
    for iteration in range(nb_itr):
        # Loop over N
        for N in N_range:
            T = np.arange(0,10, 0.2)
            nu = np.array( [complex(0.1*a,b) for a in range(0  , 10) for b in [0.01,0.1]] )
            c = 1
            p = int(c*N)

            generated_data = gen_matrix(Scenario, p)
            population_spectrum, weights, support = generated_data
            diag           = freeDeconvolution.sampling.sample_wishart( p, N, population_spectrum )

            population_cdf = np.zeros_like( T )
            for i in range( len(T) ):
                t = T[i]
                population_cdf[i] = np.count_nonzero( population_spectrum <= t )
            population_cdf = population_cdf/p
            #
            DOEs.append( {
                "Scenario": Scenario,
                "N"       : N,
                "observed_spec"    : list(diag),
                "population_spec"  : list(population_spectrum),
                "population_cdf"   : list(population_cdf),
                "truth_weights"    : list(weights),
                "truth_support"    : list(support),
                "results_by_method": {}
            } )
        #
    #
#

In [ ]:
with open("./DOEs.json", "w") as f:
    json.dump( DOEs, f, indent=4)

In [ ]:
# WARNING: Crushes old results
with open("./DOEs_with_results.json", "w") as f:
    json.dump( DOEs, f, indent=4)

# II. Loading and processing DOEs

## II.1. El Karoui's method

In [ ]:
with open("./DOEs_with_results.json", "r") as f:
    DOEs = json.load(f)
#
print( f'''Loaded {len(DOEs)} experiments...''')

In [ ]:
methods = [ "convex_optim", "subordination", "our method"]
method  = methods[0]

In [ ]:
def compute_DOE_with_convex( DOE, norm="l2"):
    c = 1
    T = np.arange(0,10, 0.2)
    nu = np.array( [complex(0.1*a,b) for a in range(0  , 10) for b in [0.01,0.1]] )

    # RMT data
    diag = np.array( DOE["observed_spec"] )
    population_spectrum = np.array( DOE["population_spec"] )
    population_cdf      = np.array( DOE["population_cdf"] )

    ## El Karoui
    tic = time.time()
    Z = freeDeconvolution.elkaroui.build_dictionary( nu, c, diag)
    nu_check = -(1-c)/Z + c*freeDeconvolution.elkaroui.stieltjes(Z, diag)
    nu_errors = np.abs(nu - nu_check)
    bad_indices = np.where(nu_errors > 1e-5)

    # Clean-up if necessary
    if len(bad_indices):
        Z  = np.delete( Z , bad_indices )
        nu = np.delete( nu, bad_indices )
    dictionary = (Z, nu)

    # Perform optimization
    weights_convex, objective_value = freeDeconvolution.elkaroui.perform_cvx_optimization( dictionary, T, c, norm, verbose=False)
    weights_convex = abs(weights_convex)
    toc    = time.time()
    timing = toc-tic
    print( f'''Timing for convex optimization by el Karoui {str(timing)}''' )

    ## Done
    error   = wasserstein_distance( T, population_spectrum,  weights_convex, np.ones(len(population_spectrum))/len(population_spectrum))
    new_DOE = DOE.copy()
    new_DOE["results_by_method"]["convex_optim"] = {
        "error": error,
        "timing": timing,
        "weights": list(weights_convex),
        "support": list(T)
    }

    return new_DOE

In [ ]:
# Loop over experiments
print( "Loop over DOEs using multiprocessing... ")
num_processes = 12 # Use all the available CPU cores of computer
with mp.Pool(processes=num_processes) as pool:
    results = pool.map( compute_DOE_with_convex , DOEs)

In [ ]:
with open("./DOEs_with_results.json", "w") as f:
    json.dump( results, f, indent=4)

## II.2. Tarrago's method

In [ ]:
with open("./DOEs_with_results.json", "r") as f:
    DOEs = json.load(f)
#
print( f'''Loaded {len(DOEs)} experiments...''')

In [ ]:
## II.2. Processing with the subordination method
methods = [ "convex_optim", "subordination"]
method  = methods[1]

In [ ]:
def compute_DOE_with_subordination( DOE):
    # RMT data
    diag = np.array( DOE["observed_spec"] )
    population_spectrum = np.array( DOE["population_spec"] )
    population_cdf      = np.array( DOE["population_cdf"] )
    
    ## Tarrago
    tic  = time.time()
    y, R = freeDeconvolution.subordination.freedeconvolutionresult(diag)
    weights_subordination = abs(R)/np.sum(R)
    toc    = time.time()
    timing = toc-tic
    print( f'''Timing for convex optimization by Tarrago {str(timing)}''' )

    ## Done
    error   = wasserstein_distance( y, population_spectrum,  weights_subordination, np.ones(len(population_spectrum))/len(population_spectrum))
    new_DOE = DOE.copy()
    new_DOE["results_by_method"]["subordination"] = {
        "error"  : error,
        "timing" : timing,
        "weights": list(weights_subordination),
        "support": list(y)
    }

    return new_DOE

In [ ]:
# Loop over experiments
print( "Loop over DOEs using multiprocessing... ")
num_processes = 12 # Use all the available CPU cores of computer
with mp.Pool(processes=num_processes) as pool:
    results = pool.map( compute_DOE_with_subordination , DOEs)

In [ ]:
with open("./DOEs_with_results.json", "w") as f:
    json.dump( results, f, indent=4)

## II.3. Our method

In [ ]:
with open("./DOEs_with_results.json", "r") as f:
    DOEs = json.load(f)
#
print( f'''Loaded {len(DOEs)} experiments...''')

In [ ]:
## II.3. Processing with our method
methods = [ "convex_optim", "subordination", "our method"]
method  = methods[2]

In [ ]:
def compute_DOE_with_our_method( DOE, debug=True, debug_aggressive=False, plot=True):
    c = 1
    N = DOE['N']
    p = int(c*N)
        
    # RMT data
    diag = np.array( DOE["observed_spec"] )
    population_spectrum = np.array( DOE["population_spec"] )
    population_cdf      = np.array( DOE["population_cdf"] )
    
    tic  = time.time()

    ## Init
    mu_observed = freeDeconvolution.core.DiscreteMeasure( diag, None)
    mu_signal   = freeDeconvolution.core.DiscreteMeasure( population_spectrum, None)

    mu_observed.compute_second_kind()

    zeroes_first_kind   = mu_observed.zeroes_first_kind
    zeroes_second_kind  = mu_observed.zeroes_second_kind

    ## Find bounding box
    from freeDeconvolution import boxes
    degree = len(diag)

    if debug:
        print("")
        print("-----------------------------------")
        print("id       : ", DOE['id'])
        print("Scenario : ", DOE['Scenario'])
        print("")

    mesh_size = 10000
    # radius = np.max(diag)/2 + 1
    # center = np.max(diag)/2
    # interval = np.linspace(0, 2*np.pi, mesh_size)
    # contour = center + radius*( np.cos(interval) + np.sin(interval)*1.0j)
    # plt.scatter( np.real(diag), np.imag(diag), c='r')
    # plt.plot( np.real(contour), np.imag(contour) )
    # plt.show()

    def index_integrand(z):
        values = mu_observed.Markov_Krein_prime(z)/mu_observed.Markov_Krein(z)
        return values

    # values = index_integrand(contour)
    # dz = 1.0j*(contour-center)*2*np.pi/(mesh_size) 
    # index  = np.sum(dz*values)/(2*np.pi*1.0j)
    # print( "Index: ", index)
    # print( "Root count: ", index+2*degree)

    # print( "Box segments enumeration: ")
    # print( boxes.box_segments_enum )

    def compute_index( box, mesh_size, plot=True, color='b'):
        interval =  np.linspace( 0,1, mesh_size)
        integral = 0
        for segment in boxes.box_segments_enum:
                vector = box[ segment[1] ] - box[ segment[0] ]
                origin = box[ segment[0] ]
                s = origin + interval*vector
                #
                values = index_integrand(s)
                dz = ( s[-1]-s[0] )/mesh_size
                integral = integral + np.sum( values*dz )
        return integral/(2*np.pi*1.0j)
    # TODO: Make it more versatile. Here tuning by hand of radius.

    radius = 4
    mesh_size = int(1e4)
    bounding_box = {
        'top_left'    : np.min(diag) - 0.3 + radius*1.0j,
        'bottom_right': np.max(diag) + 0.3 - radius*1.0j,
    }
    bounding_box   = boxes.extend_box(bounding_box)
    index = compute_index( bounding_box, mesh_size)
    index = np.real(index+2*degree)
    root_count = int( np.round( index ) )
    error = index-root_count
    if debug:
        print( "Checking the number of critical points in initial bounding box")
        print( "Index of contour : ", index)
        print( "Root count: ", "2x", 0.5*root_count)
        print( "p         : ", p)
        print( "")

    # If false, the bounding box missed roots
    assert( p-1 == int(0.5*root_count))
    
    ## Find small contour
    find_small_contour = True
    if not find_small_contour:
        if debug:
            print("Using default bounding box...")
        contour_height = radius
    else:
        if debug:
            print("Finding near-optimal bounding box...")
        #
        box = bounding_box.copy()
        box['bottom_left'] = box['top_left'] # For initialization, bottom_left needs to be the previous top_left
        radius = box['height']/2
        stop_at_first_nonempty = True        # Stop at first found box with roots

        # Loop for multiple passes and more
        boxes_with_roots = []
        boxes_with_roots.append( box )
        root_counter = 0
        total_roots  = p-1 # Total number of roots in upper half plane
        i = 0
        while( root_counter < total_roots ):
            i = i+1
            radius = radius/2
            box = {
                    'top_left'    : box['bottom_left'],
                    'bottom_right': np.real( box['bottom_right'] ) + radius*1.0j,
            }
            box   = boxes.extend_box(box)
            index = compute_index( box, mesh_size, plot=False)
            index = np.real(index)
            root_count = int( np.round( index ) )
            root_counter = root_counter + root_count
            error = index-root_count
            if debug_aggressive:
                print(f"Pass {i}:")
                print( "Index: ", index)
                print( "Root count / Total: ", root_count, '/', root_counter)
                print( "Found:", root_counter, " / ", total_roots )
                print( "")
            #
            if root_count>0:
                    box['root_count'] = root_count
                    boxes_with_roots.append( box )
                    if stop_at_first_nonempty:
                            break
        # end  while
        contour_height = 2*radius
    # end if find_small_contour

    ## Setup arrays
    contour_type = "rectangle"
    mesh_size    = int(1e4)
    left_point   = np.min(diag)
    right_point  = np.max(diag)
    mid_point    = 0.5*left_point + 0.5*right_point
    
    # New bounding box
    print( "Contour height: ", contour_height )
    box = {
            'top_left'    : np.real( bounding_box['top_left'],    ) - 1.0 + contour_height*1.0j,
            'bottom_right': np.real( bounding_box['bottom_right'] ) + 1.0 - contour_height*1.0j,
    }
    box   = boxes.extend_box(box)
    print( "Bounding box height (new): ", box )
    bounding_box = box


    if contour_type=="rectangle":
        path = boxes.box_to_path( bounding_box, mesh_size )
        z_array = np.array( path )
    elif contour_type=='circle':
        radius = np.maximum( 1.1*(right_point-left_point)/2 , bounding_box["height"] )
        #
        interval = np.linspace(0, 2*np.pi, mesh_size)
        z_array = mid_point + radius*( np.cos(interval) + np.sin(interval)*1.0j)
    
    m_array = mu_observed.M_empirical( z_array )
    s_array = (1+m_array)/(m_array*z_array)

    # Various arrays
    s_signal_array = s_array
    s_noise_array  = 1/(c*m_array + 1)
    s_deconv_array = s_signal_array/s_noise_array
    #
    # NEXT LINE IS F*** UP
    # m_deconv_array = 1/( s_deconv_array*z_array - 1)
    #
    # TENTATIVE FIX
    m_deconv_inv_array = (1+1/m_array)/s_deconv_array
    m_deconv_array   = m_array
    z_array_original = z_array
    z_array          = m_deconv_inv_array
    #
    #m_deconv_theoretical_array = mu_signal.M_empirical( z_array )
    g_deconv_array = (m_deconv_array+1)/z_array
    #g_deconv_theoretical = (m_deconv_theoretical_array+1)/z_array


    ## Plot of newly found contour
    if plot:
        plt.figure()
        plt.rcParams["figure.figsize"] = (5, 5)
        plt.plot( np.real(z_array), np.imag(z_array), label="Transformed z contour" )
        plt.plot( np.real(z_array_original), np.imag(z_array_original), label="Original z contour" )
        plt.scatter( zeroes_first_kind, np.zeros_like(zeroes_first_kind), label="Roots of $mu_n$")
        plt.title( f'''Contour for DOE id={DOE['id']}''' )
        plt.legend()
        plt.savefig(f'''contour_id_{DOE['id']}.png''' )

    ## Compute moments
    dz_array  = z_array-np.roll(z_array, shift=1)
    # Rectangle Riemann integration
    def cauchy_integral_g_deconv( f ):
        value = g_deconv_array*f*dz_array
        #value = g_deconv_theoretical*f*dz_array
        return value.sum()/(2*np.pi*1.0j)
    # Trapezoid Riemann integration
    def cauchy_integral_trapezoid_g_deconv( f ):
        integrand = g_deconv_array*f
        value = 0.5*( integrand + np.roll(integrand, shift=1) )*dz_array
        return value.sum()/(2*np.pi*1.0j)
    moments_count = 5
    mom_array       = np.zeros( 2*moments_count + 2)
    mom_array_truth = np.zeros( 2*moments_count + 2)
    truth_weights   = np.array( DOE['truth_weights'] )
    truth_support   = np.array( DOE['truth_support'] )
    centering       = 0
    for mom_index in range( len(mom_array) ):
        value = cauchy_integral_trapezoid_g_deconv( (z_array-centering)**mom_index ) # Centering stabilizes
        mom_array[ mom_index ] = np.real(value)
        # Activate centering
        # if mom_index==1:
        #      centering    = mom_array[1]
        #      mom_array[1] = 0
        mom_array_truth[mom_index] = np.dot( truth_weights, (truth_support-centering)**mom_index)
    # end for

    # Chebyshev
    endpoint_left  = np.min( np.real(z_array) )
    endpoint_right = np.max( np.real(z_array) )
    centering = 0.5*( endpoint_left + endpoint_right )
    scale     = endpoint_right - centering
    normalized_z_array = (z_array-centering)/scale
    
    chebyshev_z_array     = compute_chebyshev_values( normalized_z_array, 2*moments_count + 2)
    chebyshev_truth_array = compute_chebyshev_values( (truth_support-centering)/scale, 2*moments_count + 2)
    chebyshev_truth_array = np.real( chebyshev_truth_array )

    #print( "Chebyshev on support :", chebyshev_truth_array)

    chebyshev_mom_array       = np.zeros( 2*moments_count + 2)
    chebyshev_mom_array_truth = np.zeros( 2*moments_count + 2)
    for mom_index in range( len(mom_array) ):
        value = cauchy_integral_trapezoid_g_deconv( chebyshev_z_array[mom_index] ) # Centering stabilizes
        chebyshev_mom_array[ mom_index ] = np.real(value)
        chebyshev_mom_array_truth[mom_index] = np.dot( truth_weights, chebyshev_truth_array[mom_index] )
    # end for

    if debug:
        np.set_printoptions(precision=5, suppress=True)
        #print( "Midpoint: ", mid_point )
        print( "Estimated moments: ")
        print( mom_array )
        print( "Ground truth:")
        print( mom_array_truth )
        print( "Absolute error:")
        print( np.abs(mom_array-mom_array_truth) )
        print( "Relative error:")
        print( np.abs( (mom_array-mom_array_truth)*100/mom_array_truth ) )
        print( "")
        print( "Estimated Chebyshev moments: ")
        print( chebyshev_mom_array )
        print( "Ground truth:")
        print( chebyshev_mom_array_truth )
        print( "Absolute error:")
        print( np.abs( chebyshev_mom_array-chebyshev_mom_array_truth ) )
        print( "Relative error:")
        print( np.abs( (chebyshev_mom_array-chebyshev_mom_array_truth)*100/chebyshev_mom_array_truth ) )
        print( "")

    # WARNING: For debug
    chebyshev_mom_array = chebyshev_mom_array_truth

    ## Inverse moment problem
    print( "Performing inverse Chebychev moment problem...")
    jacobi_a, jacobi_b = jacobi_from_chebyshev_moments( chebyshev_mom_array )
    print( "a :", jacobi_a )
    print( "b :", jacobi_b )
    support, weights   = freeDeconvolution.quadrature_from_jacobi( jacobi_a, jacobi_b)
    print( "Centered support: ", support )
    print( "Weights: ", weights)
    support = scale*support + centering
    non_outliers = np.where( (support>left_point)*(support<right_point) )[0]
    support = support[non_outliers]
    weights = weights[non_outliers]
    weights = weights/np.sum(weights)
    error   = wasserstein_distance( support, population_spectrum, weights, np.ones(len(population_spectrum))/len(population_spectrum))
    
    print( "Error  : ", error)
    print( "Weights: ", weights)
    print( "Support: ", support)
    print( "" )

    ## Inverse moment problem
    print( "Performing inverse moment problem...")
    jacobi_a, jacobi_b = freeDeconvolution.oprl.jacobi_from_moments( mom_array )
    print( "a :", jacobi_a )
    print( "b :", jacobi_b )
    support, weights   = freeDeconvolution.quadrature_from_jacobi( jacobi_a, jacobi_b)
    support = support # + centering
    non_outliers = np.where( (support>left_point)*(support<right_point) )[0]
    support = support[non_outliers]
    weights = weights[non_outliers]
    weights = weights/np.sum(weights)
    error   = wasserstein_distance( support, population_spectrum, weights, np.ones(len(population_spectrum))/len(population_spectrum))
    
    print( "Error  : ", error)
    print( "Weights: ", weights)
    print( "Support: ", support)
    print( "" )

    # Ground truth
    print( "Ground truth: ")
    print( truth_weights )
    print( truth_support )
    print( "")

    # Timings
    toc    = time.time()
    timing = toc-tic
    print( f'''Timing for our method {str(timing)}''' )
    
    ## Recording
    new_DOE = DOE.copy()
    new_DOE["results_by_method"]["our method"] = {
        "error"  : error,
        "timing" : timing,
        "weights": list(weights),
        "support": list(support)
    }

    with open( f'''./dump/DOE_{DOE['id']}.json''', "w") as f:
        json.dump( new_DOE, f, indent=4)

    return new_DOE

In [ ]:
def compute_chebyshev_values( x, order ):
    result = []
    #
    term_0 = np.ones_like(x)*1.0
    result.append( term_0 )
    term_1 = x
    result.append( x )
    for i in range(2, order+1):
            # Compute next term in recurrence
            # Normally P_{n+1} = 2X P_n - P_{n-1}
            # But P_n = 2^{n-1} T_n
            # Hence T_{n+1} = X P_n - (1/4)*P_{n-1}
            if i==2:
                 b2 = 0.5
            else:
                 b2 = 0.25
            term_2 = x*term_1 - b2*term_0
            result.append( term_2 )
            # Shift variables
            term_0 = term_1 
            term_1 = term_2
    # end for
    return result

In [ ]:
# Input: Array of 2n+1 moments from c_0=1 to c_{2n+1}
def jacobi_from_chebyshev_moments( mom_array, debug=False ):
    assert( (len(mom_array)-1) % 2 == 1 )
    mom_array = np.hstack( (mom_array, [0]))
    
    # Form Gram matrix
    # Uses the fact that
    # 2 P_n P_m = P_{n+m} + P_{|n-m|} for the (non-monic) Chebyshev
    # This leads to
    # T_n T_m = T_{n+m} + 2^{|m-n|-n-m} T_{|n-m|}
    #         = T_{n+m} + 2^{-2*min(n,m)} T_{|n-m|}
    # Except when n=m where
    # T_n^2 = T_{2n} + 2^{-2n+1}
    n = int( 0.5*(len(mom_array)-2) )
    gram_matrix = np.zeros( shape=(n+2, n+2) )
    for i in range( n+2 ):
        if i==0:
            for j in range( i+1, n+2):
                gram_matrix[i,j] = mom_array[i+j]
        else:
            for j in range( i+1, n+2):
                gram_matrix[i,j] = mom_array[i+j] + (2**(-2*i))*mom_array[j-i]
    gram_matrix = gram_matrix + gram_matrix.transpose()
    for index in range(1, n+2 ):
        gram_matrix[index, index] = mom_array[2*index] + 2**(-2*index+1)
    gram_matrix[0,0] = 1

    if debug:
        print( "Gram matrix: ", gram_matrix)
    
    # Cholesky
    # try:
    #     cholesky = scipy_cholesky( mom_matrix, lower=True )
    #     print("Cholesky passed!")
    #     print("")
    # except np.linalg.LinAlgError as err:
    #     print( err  )
    #     print("")

    # Use of LAPACK wrapper in scipy
    # https://stackoverflow.com/questions/49101574/scipy-numpy-cholesky-while-checking-if-positive-definite
    (cholesky, minor) = scipy.linalg.lapack.dpotrf( gram_matrix, lower=True  )
    if minor > 0:
        # if debug:
        #     print( f'''{minor}-th principal minor is not positive definite''')
        print( f'''{minor}-th principal minor is not positive definite''')
        gram_matrix = gram_matrix[:minor, :minor]
        cholesky   = cholesky[:minor, :minor]
        #
        n = minor - 1

    if debug:
        print( "Cholesky matrix: ")
        print( cholesky )

    mom_count = cholesky.shape[0]
    diag_indices, extra_indices = freeDeconvolution.oprl.jacobi_indices( mom_count )
    diag_cholesky = cholesky.T[diag_indices[0,:] , diag_indices[1,:]]
    extra_diag    = cholesky.T[extra_indices[0,:], extra_indices[1,:]]

    if debug:
        print( "Diagonal of Cholesky")
        print( diag_cholesky )
        print( "Extra-diagonal of Cholesky")
        print( extra_diag )
        print("")

    # Compute Jacobi
    jacobi_b = diag_cholesky[1:]/diag_cholesky[:-1]
    jacobi_b = jacobi_b[:-1]
    jacobi_a = np.zeros_like( extra_diag )
    if len(extra_diag)>0:
        jacobi_a[0] = extra_diag[0]
        x_over_y = extra_diag/diag_cholesky[:-1]
        jacobi_a[1:] = x_over_y[1:]-x_over_y[:-1]

    return jacobi_a, jacobi_b

In [ ]:
import os
import string

# Tag DOEs for identification if bugs
for i in range( len(DOEs) ):
    DOE       = DOEs[i]
    DOE['id'] = 31000+i

# Lazy mode
lazy = False
if lazy:
    print( "Working in lazy mode...")
    files = os.listdir('./dump')
    str_identifiers = [ f.strip(string.ascii_letters)[1:-1] for f in files]
    identifiers     = sorted( [ int(str_id)-31000 for str_id in str_identifiers] )
    print( "Found files: ", identifiers )
    all_indices      = set( range(len(DOEs)) )
    selected_indices = all_indices.difference( identifiers )
    selected_indices = list(selected_indices)[:]
    print( f'''Number of selected indices: {len(selected_indices)}/{len(DOEs)}''')
    print( "Selected indices:", selected_indices)
    selected_DOEs = [ DOEs[i] for i in selected_indices ]
    print( "")
else:
    selected_DOEs = DOEs

# Loop over experiments
print( "Loop over DOEs using multiprocessing... ")
num_processes = 1 # Use all the available CPU cores of computer
with mp.Pool(processes=num_processes) as pool:
    results = pool.map( compute_DOE_with_our_method, selected_DOEs)
# for DOE in selected_DOEs:
#     compute_DOE_with_our_method( DOE )

In [ ]:
with open("./DOEs_with_results.json", "w") as f:
    json.dump( results, f, indent=4)

## II.4. Fusion

In [ ]:
import os
import string

files = os.listdir('./dump')
DOEs_fused = []
for filename in files:
    f = open( f'''./dump/{filename}''', 'r')
    DOE = json.load( f )
    DOEs_fused.append( DOE )
# end for

with open("./DOEs_fused.json", "w") as f:
    json.dump( DOEs_fused, f, indent=4)


# III. Plotting

In [ ]:
with open("./DOEs_with_results.json", "r") as f:
    DOEs = json.load(f)
#
print( f'''Loaded {len(DOEs)} experiments...''')

In [ ]:
methods = [ "convex_optim", "subordination", "our method"]

aggregated_results = freeDeconvolution.plots.aggregate_benchmarks( DOEs, methods)
freeDeconvolution.plots.make_plots( aggregated_results, methods )

# IV. Analysis

Here we dump data to csv file for further analysis

In [ ]:
with open("./DOEs_with_results.json", "r") as f:
    DOEs = json.load(f)
#
print( f'''Loaded {len(DOEs)} experiments...''')

In [ ]:
extracts = []
for DOE in DOEs:
    result   = DOE['results_by_method']['our method']
    error    = result['error']
    timing   = result['timing']
    extract = [ DOE['Scenario'], DOE['N'], error, timing, result["weights"], result["support"] ]
    extracts.append( extract )
# end for

import csv

col_names = ["Scenario", "N", "error", "timing", "weights", "support"]

with open("./extracts.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerow( col_names )
    writer.writerows(extracts)